In [1]:
import configparser
import datetime as dt
import sys
import collections
from pathlib import Path
from typing import Union, Optional, List, Tuple

import codebook.EDA as EDA
import codebook.clean as clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [3]:
Path.cwd().parent / 'src'

WindowsPath('C:/Users/r2d4/OneDrive/code/projects/20-02_disco/src')

In [4]:
sys.path.append(str(Path.cwd().parent))

from src.db_declaration import (
    Base, Artist, CreditTrx, Genre, Label, Record, RecordFormat, CreditTrx, LabelRecordLink, ArtistLabelLink
)
from src import db_functions
from src import db_connect

In [5]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## Connect To DB

Because we connect from `dev`subfolder, I have to work with the sqlalchemy `create_engine` function directly.

In [6]:
# session.close()
prod_path = r"sqlite:///C:\Users\r2d4\OneDrive\code\projects\20-02_disco\db_aka_discobase\DiscoBase.db"
engine = sqlalchemy.create_engine(prod_path)

session = db_connect.create_session(engine)

# Check
session

## Query DB

In [7]:
pd.read_sql("records", engine).tail(5)

,record_id,title,year,genre_id,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,rating,is_digitized,is_active,created_at,updated_at
344,345,Evil Power,2015,2,3,yellow-red-black,lim,None,None,2021-07-14,35,None,0,1,2021-07-14 18:13:49,NaT
345,346,Phantom Knights,2017,2,3,yellow-red-black,200,None,1st vinyl pressing of 2010 record,2021-07-14,30,None,1,1,2021-07-14 18:15:28,2021-07-23 16:06:41
346,347,Visions Of Trismegistos,2021,2,3,oxblood-silver swirl,lim,None,None,2021-07-18,20,None,1,1,2021-07-18 10:06:13,2021-07-28 16:38:39
347,348,Casting The Die,2021,2,3,purple,200,None,None,2021-07-18,25,None,0,1,2021-07-18 10:08:22,NaT
348,349,Dawn of the Primitive Age,2021,2,3,orange,100,None,None,2021-07-30,20,None,1,1,2021-07-30 18:45:57,NaT


In [8]:
pd.read_sql("records", engine)["is_active"].sum()

315

In [9]:
pd.read_sql("artists", engine).tail

<bound method NDFrame.tail of      artist_id                artist_name artist_country          created_at          updated_at
0            1                  Dismember         Sweden 2021-02-24 14:24:16                 NaT
1            2                  Merciless         Sweden 2021-02-24 14:24:16 2021-02-28 16:57:22
2            3                   Overkill            USA 2021-02-24 14:24:16 2021-02-24 14:44:37
3            4                   Entombed         Sweden 2021-02-24 14:24:16                 NaT
4            5                  Metallica            USA 2021-02-24 14:24:16 2021-02-28 16:59:06
..         ...                        ...            ...                 ...                 ...
223        224                 Mortiferum            USA 2021-06-19 18:22:38                 NaT
224        225                Deathhammer         Norway 2021-07-14 18:13:49                 NaT
225        226              Nekromantheon         Norway 2021-07-18 10:06:13                 NaT


In [10]:
pd.read_sql("credit_trx", engine).tail(10)

,credit_trx_id,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
375,376,2021-07-02,Purchase,-1.00,-5.00,343.00,2021-07-03 20:05:42,NaT
376,377,2021-07-02,Purchase,0.00,-5.00,344.00,2021-07-03 20:06:45,NaT
377,378,2021-07-08,Addition,1.00,-4.00,nan,2021-07-14 18:11:01,NaT
378,379,2021-07-14,Purchase,-1.00,-5.00,345.00,2021-07-14 18:13:49,NaT
379,380,2021-07-14,Purchase,-1.00,-6.00,346.00,2021-07-14 18:15:28,NaT
380,381,2021-07-18,Addition,1.00,-5.00,nan,2021-07-18 10:01:56,NaT
381,382,2021-07-18,Purchase,-1.00,-6.00,347.00,2021-07-18 10:06:13,NaT
382,383,2021-07-18,Purchase,-1.00,-7.00,348.00,2021-07-18 10:08:22,NaT
383,384,2021-07-28,Addition,1.00,-6.00,nan,2021-07-28 16:38:07,NaT
384,385,2021-07-30,Purchase,-1.00,-7.00,349.00,2021-07-30 18:45:57,NaT


In [11]:
pd.read_sql("record_label_link", engine).tail()
pd.read_sql("artist_label_link", engine).tail()
pd.read_sql("artist_genre_link", engine).tail()
pd.read_sql("genre_label_link", engine).tail()

,record_id,label_id
303,304,25
304,305,33
305,306,33
306,307,33
307,308,45


,artist_id,label_id
333,151,16
334,225,30
335,226,30
336,227,116
337,228,117


,artist_id,genre_id
226,224,1
227,225,2
228,226,2
229,227,2
230,228,2


,genre_id,label_id
125,1,114
126,1,115
127,2,30
128,2,116
129,2,117


In [12]:
pd.read_sql("labels", engine).sort_values("label_name")

,label_id,label_name,created_at,updated_at
17,18,20 Buck Spin,2021-02-24 14:24:22,NaT
27,28,6131,2021-02-24 14:24:23,NaT
89,90,Abfall,2021-02-28 17:49:06,NaT
91,92,Acouphènes!,2021-02-28 17:50:46,NaT
9,10,Active Rebellion,2021-02-24 14:24:20,NaT
...,...,...,...,...
49,50,Vinyl Collectors,2021-02-24 16:31:31,NaT
73,74,Virgin,2021-02-28 11:24:25,NaT
69,70,War Anthem,2021-02-28 10:48:59,NaT
64,65,Witching Hour Productions,2021-02-26 17:07:08,NaT


In [13]:
result = session.query(Record).filter(Record.artists.any(Artist.artist_name == "Entombed")).all()
for record in result:
    print(record, record.purchase_date)

<Record(record_id=11, title=Uprising, artist=[<Artist(artist_id=4, artist_name=Entombed)>])> 2013-07-01
<Record(record_id=12, title=Morning Star, artist=[<Artist(artist_id=4, artist_name=Entombed)>])> 2013-07-01
<Record(record_id=23, title=Wolverine Blues, artist=[<Artist(artist_id=4, artist_name=Entombed)>])> 2014-01-01
<Record(record_id=36, title=Clandestine, artist=[<Artist(artist_id=4, artist_name=Entombed)>])> 2015-08-01
<Record(record_id=40, title=Left Hand Path, artist=[<Artist(artist_id=4, artist_name=Entombed)>])> 2015-10-01


C:\Users\r2d4\miniconda3\envs\py3\lib\site-packages\sqlalchemy\sql\sqltypes.py:660: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


In [14]:
result = session.query(Artist).filter(Artist.artist_name == "Entombed").one()
print(result.artist_country)

Sweden


In [15]:
pd.read_sql("formats", engine)

,format_id,format_name,created_at,updated_at
0,1,"12""",2021-02-24 14:24:16,NaT
1,2,"Pic-7""",2021-02-24 14:24:16,2021-02-24 14:37:30
2,3,LP,2021-02-24 14:24:16,NaT
3,4,2LP,2021-02-24 14:24:16,NaT
4,5,"10""",2021-02-24 14:24:16,NaT
5,6,"7""",2021-02-24 14:24:16,NaT
6,7,MLP,2021-02-24 14:24:17,NaT
7,8,2xLP,2021-02-24 14:24:17,NaT
8,9,LP,2021-02-24 14:24:20,NaT
9,10,Pic-LP,2021-02-24 14:24:21,NaT


In [16]:
pd.read_sql("genres", engine)

,genre_id,genre_name,created_at,updated_at
0,1,Death Metal,2021-02-24 14:24:16,NaT
1,2,Thrash Metal,2021-02-24 14:24:16,NaT
2,3,Black Metal,2021-02-24 14:24:16,NaT
3,4,Speed Metal,2021-02-24 14:24:17,NaT
4,5,Hardcore,2021-02-24 14:24:18,NaT
5,6,Punk,2021-02-24 14:24:19,NaT
6,7,Crust,2021-02-24 14:24:19,NaT
7,8,Crossover,2021-02-24 14:24:24,NaT
8,9,Grindcore,2021-02-24 14:24:25,NaT


## Inactivate A Record (Temporary Section)

In [17]:
# removal = {
#     "trx_type": "Removal",
#     "credit_value": 0,
#     "artist": "Furia",
#     "title": "Marzannie, Królowej Polski",
#     "year": 2012,
#     "date": dt.datetime.today().date()
# }

In [18]:
# db_functions.set_record_to_inactive(session, removal)

In [19]:
# pd.read_sql("credit_trx", engine).tail()

In [20]:
# to_delete = session.query(CreditTrx).filter(CreditTrx.credit_trx_id == 301).one()
# session.delete(to_delete)
# session.commit()

## Backfill Missing Record-Format-Relations in Records Table (Done)

In [21]:
# def load_albums_from_xlsx(
#     filepath: Union[Path, str], genres: Optional[List] = None
# ) -> pd.DataFrame:
#     """Load the original album collection file into a dataframe.
#     You can specify a list of genres you want to include
#     (defaults to None).
#     """
#     df = pd.read_excel(filepath, engine="openpyxl")
#     if genres:
#         df = df[df["Genre"].isin(genres)]
#     return df

In [22]:
# CONFIG_PATH = Path.cwd().parent / "config.cfg"
# path_to_collection = utils.read_config_return_str(CONFIG_PATH, "COLLECTION")
# collection_df = load_albums_from_xlsx(path_to_collection)
# collection_df = clean.prettify_column_names(collection_df)
# collection_df.dropna(thresh=8, inplace=True)
# collection_df.rename(columns={"format": "record_format"}, inplace=True)
# # collection_df["purchase_date"] = collection_df["purchase_date"].dt.date.astype(str)
# # collection_df["number"] = collection_df["number"].astype(str)
# # collection_df["digitized"] = collection_df["digitized"].map({True: 1, False: 0}).astype(int)
# collection_df["label"].fillna("NA", inplace=True)  # because it creates a foreign key

# # This is for the initial ingestion only
# collection_df["artist_country"] = None
# collection_df["credit_value"] = 0
# collection_df["trx_type"] = "Initial Load"

# assert len(collection_df) == 293

In [23]:
# # backfill

# for rec in collection_df.itertuples():
#     record = db_functions.fetch_a_record_from_the_shelf(session, rec.artist, rec.title)
#     r_format = rec.record_format
    
#     # Check if the format already exists or has to be created
#     record_format = (
#         session.query(RecordFormat)
#         .filter(RecordFormat.format_name.ilike(r_format))
#         .one_or_none()
#     )
#     if record_format is None:
#         record_format = RecordFormat(format_name=r_format)
#         session.add(record_format)
    
#     record.record_format = record_format

# session.commit()

In [24]:
# # Manual Update

# record = session.query(Record).filter(Record.record_id == 295).one()
# r_format = '12"'

# # Check if the format already exists or has to be created
# record_format = (
#     session.query(RecordFormat)
#     .filter(RecordFormat.format_name.ilike(r_format))
#     .one_or_none()
# )
# if record_format is None:
#     record_format = RecordFormat(format_name=r_format)
#     session.add(record_format)

# record.record_format = record_format

# session.commit()

## Manual Updates

### Credit Trx

In [25]:
# result = session.query(CreditTrx).filter(CreditTrx.credit_trx_type == "Removal").all()
# for r in result:
#     r.credit_trx_type = "Remove"
#     print(r)

In [26]:
# r = session.query(Artist).filter(Artist.artist_name == "CELESTIAL SANCTUARY").one()
# r.artist_name = "Celestial Sanctuary"

In [27]:
# session.commit()

### Remove "Records" From Label-Names And Split Labels

In [28]:
# result = session.query(Label).filter(Label.label_name.like('% Records')).all()
# for x in result:
#     x.label_name = x.label_name.rpartition(" Records")[0]
# session.commit()

In [29]:
# session.query(Label).get(42)

In [30]:
# result = session.query(Label).filter(Label.label_name.contains(' / ')).all()
# for x in result:
#     print(x)

In [31]:
# result = session.query(Record).filter(Record.labels.any(Label.label_id.in_([19, 31, 32])))
# for x in result:
#     print(x)

In [32]:
# jack = session.query(Label).filter(Label.label_id.in_([19, 31, 32])).all()
# for x in jack:
#     print(x)
#     session.delete(x)
# session.commit()

In [33]:
# session.close()

### Check for "None" str Values in Ratings and Fix Them

In [34]:
# result = session.query(Record).filter(Record.rating == "None").all()
# for record in result:
#     print(record)
#     record.rating = None
# session.commit()

### Update Stuff

In [34]:
result = session.query(Record).filter(Record.artists.any(Artist.artist_country == "NA")).first()
result

<Record(record_id=275, title=Only Self, artist=[<Artist(artist_id=174, artist_name=Jesus Piece)>])>

In [36]:
# result = session.query(Artist).filter(Artist.artist_name == "Insurgency").first()
# result.artist_name = "Resurgency"
# session.commit()

In [41]:
## Replace a faulty Artist with an existing one

# session.query(Artist).filter(Artist.artist_name == "Dawn of the Primitive Age").first()
# artist = session.query(Artist).filter(Artist.artist_name == "Nuclear Revenge").first()

# result = session.query(Record).filter(Record.artists.any(Artist.artist_name == "Dawn of the Primitive Age")).all()
# result.artists[0] = artist
# session.commit()

## Delete the faulty Artist
# false_artist = session.query(Artist).filter(Artist.artist_name == "Dawn of the Primitive Age").first()
# session.delete(false_artist)
# session.commit()

In [37]:
# session.close()
# session = db_connect.create_session(engine)

In [38]:
result = session.query(Record).filter(Record.vinyl_color == 'None').all()
len(result)

0

### Test Some Queries

In [63]:
# result = (
#     session.query(Record)
#     .filter(Record.purchase_date >= '2020-06-01', Record.is_active == 1)
#     .order_by(Record.purchase_date.desc())
#     .all()
# )
# print(f"Record Ratings, {str(dt.date.today())}\n")
# for record in result:
#     print(
#         ", ".join(
#             [
#                 str(record.purchase_date),
#                 record.genre.genre_name,
#                 "/".join([x.artist_name for x in record.artists]),
# #                 record.artists[0].artist_name, 
#                 record.title, 
#                 record.record_format.format_name,
#                 str(record.rating)
#             ]
#         )
#     )

In [24]:
Record.__dict__

mappingproxy({'__module__': 'src.db_declaration',
              '__tablename__': 'records',
              'record_id': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1a17bd5a450>,
              'title': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1a17bd5a4f0>,
              'year': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1a17bd5a590>,
              'genre_id': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1a17bd5a630>,
              'format_id': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1a17bd5a6d0>,
              'vinyl_color': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1a17bd5a770>,
              'lim_edition': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1a17bd5a810>,
              'number': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1a17bd5a8b0>,
              'remarks': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1a17bd5a950>,
              'purchase_date': <sqlalchemy.orm.attributes.Instru